# building cross-compiler GNU toolchain

In [6]:
cd msp430 ; pwd ; ls

bash: cd: msp430: Нет такого файла или каталога
/home/dpon/jupy/msp430
cross.ipynb  Makefile


In [8]:
touch Makefile ; ls -la

итого 16
drwxr-xr-x  3 dpon dpon 4096 май 29 15:21 .
drwxr-xr-x 11 dpon dpon 4096 май 29 15:22 ..
-rw-r--r--  1 dpon dpon  724 май 29 15:04 cross.ipynb
drwxr-xr-x  2 dpon dpon 4096 май 29 14:41 .ipynb_checkpoints
-rw-r--r--  1 dpon dpon    0 май 29 15:28 Makefile


In [11]:
%writefile Makefile
fsdfsdf

bash: fg: %writefile: нет такого задания
bash: fsdfsdf: команда не найдена


: 127